## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
import matplotlib.pyplot as plt

# supress warnings
import warnings
warnings.filterwarnings('ignore')

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# set predictors and dependant variable
y = model_data['Angaus']
x = model_data.iloc[:, 1:]

# make copy to encode
x_encoded = x.copy()

# encode all categorical variables
for column in x.select_dtypes(include = 'object'): 
    x_encoded[column] = LabelEncoder().fit_transform(x.select_dtypes(include = 'object'))

# split data
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size = 0.2, random_state = 808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [3]:
# make first xgboost model
boost_maker = (XGBClassifier(n_estimators = 1000, 
                            learning_rate = 0.1, 
                            eval_metric = "logloss", # measure of uncertainty where 0 equals perfect model
                            early_stopping_rounds = 50, 
                            random_state = 808,
                            n_jobs = -1) # use more cores to run this function
               .fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = False)) # fit model

In [4]:
print(f"Most optimal tree/last tree where the model experienced improvement: tree {boost_maker.best_iteration}")

Most optimal tree/last tree where the model experienced improvement: tree 34


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [5]:
# make predictor grid
param_grid1 = dict(learning_rate = uniform.rvs(size = 100, scale = 0.31, loc = 0.01, random_state = 808))

# test parameters for optimal values
random_search = RandomizedSearchCV(boost_maker, 
                                   param_grid1,
                                   n_iter = 20, 
                                   random_state = 808, 
                                   n_jobs = -1
                                  )

# fit on best values
random_search.fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = 0);

In [6]:
print(f"Best learning rate: {random_search.best_params_['learning_rate']}")
print(f"Last tree where the model experienced improvement: tree {boost_maker.best_iteration}")

Best learning rate: 0.05077641254647287
Last tree where the model experienced improvement: tree 34


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [7]:
# remake boosted model with optimal values
boost_maker_op = (XGBClassifier(n_estimators = boost_maker.best_iteration, 
                                learning_rate = random_search.best_params_['learning_rate'], 
                                eval_metric = "logloss", 
                                early_stopping_rounds = 50, 
                                random_state = 808,
                                n_jobs = -1)
                   .fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = False))

In [8]:
# make new grid for new parameters
param_grid = dict(max_depth = np.arange(3, 11), 
                  min_child_weight = np.arange(1, 11), 
                  gamma = uniform.rvs(size = 100, loc = 0.05, scale = 0.10 - 0.05, random_state = 808))

In [9]:
# test new parameters for optimal values
random_search2 = RandomizedSearchCV(boost_maker_op, 
                                    param_grid,
                                    n_iter = 20, 
                                    random_state = 808, 
                                    n_jobs = -1
                                  )

# fit on best values
random_search2.fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = 0);

In [10]:
print(f"Optimal minimum number of samples needed to split: {random_search2.best_params_['min_child_weight']}")
print(f"Optimal naximum tree depth: {random_search2.best_params_['max_depth']}")
print(f"Optimal loss reduction value: {random_search2.best_params_['gamma']}")

Optimal minimum number of samples needed to split: 4
Optimal naximum tree depth: 7
Optimal loss reduction value: 0.05396814988469069


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [11]:
# remake boosted model with additional values
boost_maker_op = (XGBClassifier(**{**random_search.best_params_, **random_search2.best_params_}, 
                                eval_metric = "logloss", 
                                early_stopping_rounds = 50, 
                                random_state = 808,
                                n_jobs = -1)
                   .fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = False))

In [12]:
# make third grid with new parameters
param_dict = {
    "subsample": uniform.rvs(size = 100, loc = 0.5, scale = 0.1 - 0.05), 
    "colsample_bytree": uniform.rvs(size = 100, loc = 0.5, scale = 0.1 - 0.05)
}

In [13]:
# test parameters for optimal values
random_search3 = RandomizedSearchCV(boost_maker_op, 
                                    param_dict,
                                    n_iter = 20, 
                                    random_state = 808, 
                                    n_jobs = -1
                                  )

# fit on best values
random_search3.fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = 0);

In [14]:
# join all optimal parameter values in single dict
join = {**random_search.best_params_, **random_search2.best_params_, **random_search3.best_params_}

# print all optimal values
for feature in list(join): 
    print(f"Optimal value for {feature}: {join[feature]}")

Optimal value for learning_rate: 0.05077641254647287
Optimal value for min_child_weight: 4
Optimal value for max_depth: 7
Optimal value for gamma: 0.05396814988469069
Optimal value for subsample: 0.5394905306640807
Optimal value for colsample_bytree: 0.5348155579513307


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [15]:
# make most optimal model
boost_maker_op_final = (XGBClassifier(**join, 
                                eval_metric = "logloss", 
                                early_stopping_rounds = 50, 
                                random_state = 808,
                                n_jobs = -1)
                   .fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = False))

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [16]:
# generate probabilities
boost_probs = boost_maker_op_final.predict_proba(x_test)[:,1]

print(f"Area under curve score: {roc_auc_score(y_test, boost_probs)}")

Area under curve score: 0.8827701293283271


In [17]:
(
    pd.DataFrame(
        dict(
            Feature = boost_maker_op_final.feature_names_in_, 
            Importance = boost_maker_op_final.feature_importances_)
    )
         .sort_values('Importance', ascending = False) # sort by feature importance
         .reset_index(drop = True)
    
)

,Feature,Importance
0,SegSumT,0.168140
1,Method,0.126276
2,USNative,0.095213
3,USRainDays,0.092531
4,DSMaxSlope,0.083089
5,DSDam,0.075240
6,USSlope,0.073146
7,DSDist,0.065307
8,SegTSeas,0.058624
9,LocSed,0.057718


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

It seems that their model performs better, as it's ROC is higher. However, another way to test performance is to see how it does on new data, as in which model is more generalized. In this regard my model likely wins, as their's is more likely to have been overfit.